Yapılacaklar:
modelin aşırı öğrenmesine izin verip başarı oranını ne kadar yükselebileceğine bakmak.

### Importlar

In [16]:
import pandas as pd
from collections import Counter

import os

import shutil
import torch
import torchvision

import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

from torchvision.models import densenet161, DenseNet161_Weights

from torchvision.models import efficientnet_b2, EfficientNet_B2_Weights

from sklearn.metrics import classification_report

import copy

# import intel_extension_for_pytorch as ipex


# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt

### Cihaz tercihi

In [15]:
# input_choice = 1
# input_choice = input("Seçim yapın:\n1. CPU için\n2. GPU (CUDA) için\n3. XPU (Intel ARC) için\n (Enter'a basmak CPU seçimi yapar)\n(1/2/3): ")
# if input_choice == '1':
#     device = torch.device("cpu")
# elif input_choice == '2':
#     device = torch.device("cuda")

# elif input_choice == '3':
#     device = torch.device("xpu") 


device = torch.device("cuda")
# device = torch.device("cuda" if torch.xpu.is_available() else "cpu")

In [59]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # grayscale için
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [60]:
data_dir = 'data_model/'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
    for x in ['train', 'validation', 'test']
}

class_names = image_datasets['train'].classes
print("Sınıflar:", class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']


# Fonksiyonlar

### Model eğitecek fonksiyon

In [61]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=50, patience=5, class_names=None):
    """
    Derin öğrenme modelini eğitim ve doğrulama verisi üzerinde eğitir.

    Args:
        model (nn.Module): Eğitilecek PyTorch modeli.
        dataloaders (dict): 'train' ve 'validation' anahtarlarına sahip DataLoader sözlüğü.
        criterion (loss): Kayıp fonksiyonu.
        optimizer (torch.optim): Optimizasyon algoritması.
        device (torch.device): Eğitimde kullanılacak cihaz.
        num_epochs (int, optional): Maksimum eğitim dönemi sayısı. Varsayılan 50.
        patience (int, optional): Early stopping için tekrar süresi (doğruluk gelişmezse durma). Varsayılan 5.
        class_names (list, optional): Sınıf isimlerinin listesi. Sınıf bazlı doğruluk ve rapor için kullanılır. Varsayılan None.

    Returns:
        model (nn.Module): En iyi doğrulama başarımına sahip ağırlıklarla geri dönen model.
    """
    best_model_wts = model.state_dict()
    best_acc = 0.0
    epochs_no_improve = 0  # Early stopping için sayaç

    if class_names:
        print("Sınıflar:", class_names)
        num_classes = len(class_names)


    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)


        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()   # Eğitim moduna geç
            else:
                model.eval()    # Değerlendirme moduna geç

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []


            # Her batch için döngü
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                if phase == 'validation':
                    all_labels.extend(labels.cpu().numpy())
                    all_preds.extend(preds.cpu().numpy())


            # Epoch sonu istatistikleri
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # En iyi doğrulama modelini kaydet
            if phase == 'validation':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0  # İyileşme varsa sıfırla
                else:
                    epochs_no_improve += 1  # İyileşme yoksa artır
                print(f'{epochs_no_improve} dönem boyunca iyileşme yok.')


                # Sınıf bazlı başarı hesaplama
                if class_names:
                    class_correct = list(0. for i in range(num_classes))
                    class_total = list(0. for i in range(num_classes))

                    for i in range(len(all_labels)):
                        label = all_labels[i]
                        pred = all_preds[i]
                        if label == pred:
                            class_correct[label] += 1
                        class_total[label] += 1


                    print('\nSınıf Bazında Doğruluk:')
                    for i in range(num_classes):
                        if class_total[i] > 0:
                            accuracy = 100 * class_correct[i] / class_total[i]
                            print(f'Sınıf {class_names[i]}: {accuracy:.2f}% ({int(class_correct[i])}/{int(class_total[i])})')
                        else:
                            print(f'Sınıf {class_names[i]}: Hiç örnek bulunamadı.')


                    # Sınıf bazında rapor
                    print('\nSınıf Bazında Rapor:')
                    print(classification_report(all_labels, all_preds))

                else:
                    pass

        # Early stopping kontrolü
        if epochs_no_improve >= patience:
            print(f"\nErken durdurma tetiklendi. Doğrulama doğruluğu {patience} dönemi boyunca gelişmedi.")
            break

    # En iyi ağırlıkları geri yükle
    model.load_state_dict(best_model_wts)
    return model

### Modeli kaydet

In [ ]:
def save_model(model, model_name):
    model_save_path = f"{model_name}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model başarıyla kaydedildi: {model_save_path}")

## Örnek CNN modeli

In [62]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (1, 512, 512) -> (16, 256, 256)
        x = self.pool(F.relu(self.conv2(x)))  # (16, 256, 256) -> (32, 128, 128)
        x = self.pool(F.relu(self.conv3(x)))  # (32, 128, 128) -> (64, 64, 64)
        x = x.view(-1, 64 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## CustomCNN Model

In [63]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # -> (32, 224, 224)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (32, 112, 112)

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (64, 56, 56)

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (128, 28, 28)

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # -> (256, 14, 14)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


### Modeli oluşturma

In [64]:
scratch_model = CustomCNN(num_classes=len(class_names)).to(device)

### Loss ve optimizer

In [65]:
scratch_criterion = nn.CrossEntropyLoss()
scratch_optimizer = torch.optim.Adam(scratch_model.parameters(), lr=0.001)

# Resnet18

In [66]:
resnet18_model = models.resnet18(pretrained=True)

# İlk katman RGB (3 kanal) bekliyor. 1 kanala uyarladık.
resnet18_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Son katmanı sınıf sayısına göre değiştirdik.
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, len(class_names))

resnet18_model = resnet18_model.to(device)

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [67]:
resnet18_criterion = nn.CrossEntropyLoss().to(device)
resnet18_optimizer = optim.Adam(resnet18_model.parameters(), lr=0.001)

# DenseNet161

In [17]:
# Ağırlıkları al
densenet161_weights = DenseNet161_Weights.DEFAULT
densenet161_model = densenet161(weights=densenet161_weights)

# Giriş katmanını 1 kanala uyarlıyoruz
densenet161_model.features.conv0 = nn.Conv2d(1, 96, kernel_size=7, stride=2, padding=3, bias=False)

# Çıkış katmanını sınıf sayısına göre değiştiriyoruz
densenet161_num_ftrs = densenet161_model.classifier.in_features
densenet161_model.classifier = nn.Linear(densenet161_num_ftrs, len(class_names))

# Cihaza gönder
densenet161_model = densenet161_model.to(device)

# Loss ve optimizer
densenet161_criterion = nn.CrossEntropyLoss().to(device)
densenet161_optimizer = optim.Adam(densenet161_model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to C:\Users\keser/.cache\torch\hub\checkpoints\densenet161-8d451a50.pth


100%|██████████| 110M/110M [00:54<00:00, 2.12MB/s] 


# EfficientNet-B2

In [18]:
# En güncel ağırlıkları yükle
efficientnet_b2_weights = EfficientNet_B2_Weights.DEFAULT
efficientnet_b2_model = efficientnet_b2(weights=efficientnet_b2_weights)

# Grayscale (1 kanal) girişe uygun hale getir (ilk katman 3 kanallı bekliyor)
# efficientnet.features[0][0] => ilk Conv2d katmanı
efficientnet_b2_model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)

# Son katmanı sınıf sayısına göre ayarla
efficientnet_b2_num_ftrs = efficientnet_b2_model.classifier[1].in_features
efficientnet_b2_model.classifier[1] = nn.Linear(efficientnet_b2_num_ftrs, len(class_names))

# Cihaza gönder
efficientnet_b2_model = efficientnet_b2_model.to(device)

# Loss ve optimizer
efficientnet_b2_criterion = nn.CrossEntropyLoss()
efficientnet_b2_optimizer = optim.Adam(efficientnet_b2_model.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to C:\Users\keser/.cache\torch\hub\checkpoints\efficientnet_b2_rwightman-c35c1473.pth


100%|██████████| 35.2M/35.2M [00:21<00:00, 1.68MB/s]


In [19]:
print(efficientnet_b2_model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

# Eğitime Başlatma

In [ ]:
duraklama noktası

In [51]:
resnet18_trained_model = train_model(resnet18_model, dataloaders, resnet18_criterion, resnet18_optimizer, device, num_epochs=50, patience=10, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/50
--------------------
Train Loss: 2.3302 Acc: 0.1324
Validation Loss: 2.2661 Acc: 0.1580
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 22.00% (11/50)
Sınıf Cardiomegaly: 12.00% (6/50)
Sınıf Consolidation: 6.00% (3/50)
Sınıf Effusion: 28.00% (14/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 6.00% (3/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 84.00% (42/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.12      0.22      0.16        50
           1       0.38      0.12      0.18        50
           2       0.50      0.06      0.11        50
           3       0.25      0.28      0.26        50
           4       0.00      0.00      0.00        50
           5      

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.2679 Acc: 0.1538
Validation Loss: 2.2753 Acc: 0.1620
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 38.00% (19/50)
Sınıf Cardiomegaly: 10.00% (5/50)
Sınıf Consolidation: 56.00% (28/50)
Sınıf Effusion: 16.00% (8/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 22.00% (11/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 2.00% (1/50)
Sınıf Pneumothorax: 18.00% (9/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.11      0.38      0.18        50
           1       0.71      0.10      0.18        50
           2       0.18      0.56      0.27        50
           3       0.16      0.16      0.16        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.14      0.22      0.17        50
           7       0.00      0.00      0.00        50
           8       0.33      0.02      0.04 

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.2441 Acc: 0.1702
Validation Loss: 2.2808 Acc: 0.1520
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 62.00% (31/50)
Sınıf Effusion: 16.00% (8/50)
Sınıf Infiltration: 2.00% (1/50)
Sınıf Mass: 2.00% (1/50)
Sınıf No Finding: 4.00% (2/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 14.00% (7/50)
Sınıf Pneumothorax: 52.00% (26/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.18      0.62      0.27        50
           3       0.12      0.16      0.14        50
           4       0.04      0.02      0.03        50
           5       0.17      0.02      0.04        50
           6       0.29      0.04      0.07        50
           7       0.00      0.00      0.00        50
           8       0.09      0.14      0.11    

...

Train Loss: 2.0846 Acc: 0.2494
Validation Loss: 2.1393 Acc: 0.2480
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 12.00% (6/50)
Sınıf Cardiomegaly: 64.00% (32/50)
Sınıf Consolidation: 30.00% (15/50)
Sınıf Effusion: 30.00% (15/50)
Sınıf Infiltration: 12.00% (6/50)
Sınıf Mass: 44.00% (22/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 18.00% (9/50)
Sınıf Pleural_Thickening: 18.00% (9/50)
Sınıf Pneumothorax: 20.00% (10/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.30      0.12      0.17        50
           1       0.36      0.64      0.46        50
           2       0.26      0.30      0.28        50
           3       0.26      0.30      0.28        50
           4       0.12      0.12      0.12        50
           5       0.23      0.44      0.30        50
           6       0.00      0.00      0.00        50
           7       0.25      0.18      0.21        50
           8       0.13      0.18      

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 1.9725 Acc: 0.2947
Validation Loss: 2.1817 Acc: 0.2400
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 14.00% (7/50)
Sınıf Cardiomegaly: 74.00% (37/50)
Sınıf Consolidation: 28.00% (14/50)
Sınıf Effusion: 4.00% (2/50)
Sınıf Infiltration: 4.00% (2/50)
Sınıf Mass: 68.00% (34/50)
Sınıf No Finding: 6.00% (3/50)
Sınıf Nodule: 8.00% (4/50)
Sınıf Pleural_Thickening: 6.00% (3/50)
Sınıf Pneumothorax: 28.00% (14/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.26      0.14      0.18        50
           1       0.37      0.74      0.50        50
           2       0.22      0.28      0.25        50
           3       0.25      0.04      0.07        50
           4       0.14      0.04      0.06        50
           5       0.18      0.68      0.29        50
           6       0.43      0.06      0.11        50
           7       0.36      0.08      0.13        50
           8       0.27      0.06      0.10 

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 1.9326 Acc: 0.3075
Validation Loss: 2.0902 Acc: 0.2820
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 24.00% (12/50)
Sınıf Cardiomegaly: 72.00% (36/50)
Sınıf Consolidation: 16.00% (8/50)
Sınıf Effusion: 34.00% (17/50)
Sınıf Infiltration: 22.00% (11/50)
Sınıf Mass: 52.00% (26/50)
Sınıf No Finding: 26.00% (13/50)
Sınıf Nodule: 2.00% (1/50)
Sınıf Pleural_Thickening: 4.00% (2/50)
Sınıf Pneumothorax: 30.00% (15/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.22      0.24      0.23        50
           1       0.45      0.72      0.55        50
           2       0.23      0.16      0.19        50
           3       0.40      0.34      0.37        50
           4       0.14      0.22      0.17        50
           5       0.27      0.52      0.36        50
           6       0.25      0.26      0.26        50
           7       0.25      0.02      0.04        50
           8       0.22      0.04     

In [73]:
save_model(resnet18_trained_model, 'resnet18_model.pth')

Model başarıyla kaydedildi: resnet18_model.pth.pth


In [74]:
scratch_trained_model = train_model(scratch_model, dataloaders, scratch_criterion, scratch_optimizer, device, num_epochs=50, patience=10, class_names=class_names)

Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']

Epoch 1/50
--------------------
Train Loss: 2.3031 Acc: 0.1004
Validation Loss: 2.3053 Acc: 0.0960
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 2.00% (1/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 94.00% (47/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.09      0.02      0.03        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3040 Acc: 0.0982
Validation Loss: 2.3026 Acc: 0.1000
0 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 100.00% (50/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.10      1.00      0.18        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.00      0.00      0.00       

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3028 Acc: 0.0931
Validation Loss: 2.3026 Acc: 0.1000
1 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 100.00% (50/50)
Sınıf Pleural_Thickening: 0.00% (0/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.10      1.00      0.18        50
           8       0.00      0.00      0.00       

...

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3028 Acc: 0.0914
Validation Loss: 2.3026 Acc: 0.1000
9 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 100.00% (50/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.10      1.00      0.18       

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

Train Loss: 2.3028 Acc: 0.0969
Validation Loss: 2.3026 Acc: 0.1000
10 dönem boyunca iyileşme yok.

Sınıf Bazında Doğruluk:
Sınıf Atelectasis: 0.00% (0/50)
Sınıf Cardiomegaly: 0.00% (0/50)
Sınıf Consolidation: 0.00% (0/50)
Sınıf Effusion: 0.00% (0/50)
Sınıf Infiltration: 0.00% (0/50)
Sınıf Mass: 0.00% (0/50)
Sınıf No Finding: 0.00% (0/50)
Sınıf Nodule: 0.00% (0/50)
Sınıf Pleural_Thickening: 100.00% (50/50)
Sınıf Pneumothorax: 0.00% (0/50)

Sınıf Bazında Rapor:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        50
           1       0.00      0.00      0.00        50
           2       0.00      0.00      0.00        50
           3       0.00      0.00      0.00        50
           4       0.00      0.00      0.00        50
           5       0.00      0.00      0.00        50
           6       0.00      0.00      0.00        50
           7       0.00      0.00      0.00        50
           8       0.10      1.00      0.18      

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [75]:
save_model(scratch_trained_model, 'scratch_model.pth')

Model başarıyla kaydedildi: scratch_model.pth.pth


In [70]:
densenet161_trained_model = train_model(densenet161_model, dataloaders, densenet161_criterion, densenet161_optimizer, device, num_epochs=50, patience=10, class_names=class_names)

## Modeli yniden oluştur

In [71]:
# model = SimpleCNN(num_classes=len(class_names))  # Model sınıfınızı yeniden tanımlayın
# model.load_state_dict(torch.load("trained_model.pth"))
# model = model.to(device)
# model.eval()  # Modeli değerlendirme moduna alın
# print("Model başarıyla yüklendi.")

## Modeli Kullanma

In [72]:
# with torch.no_grad():
#     inputs = ...  # Yeni bir veri
#     inputs = inputs.to(device)
#     outputs = model(inputs)
#     _, preds = torch.max(outputs, 1)
#     print(f"Tahmin edilen sınıf: {class_names[preds[0]]}")